# Profitable App Profiles for App Store and Google Play Markets

I am completing this project under the assumption that I am working for a company that builds Android and iOS mobile apps that are available on Google Play and the App Store. 

The company only builds apps that are free to download and install, and the main source of revenue consists of in-app ads. Therefore, revenue for a given app is mostly influenced by the number of users who see and engage with the ads. 

The goal of this project is to analyze data to understand what type of apps are likely to attract more users on Google Play and the App Store.

I will be analyzing a data set containing about 10,000 [Android apps](https://www.kaggle.com/lava18/google-play-store-apps) from Google Play (collected in Aug 2018) and 7,000 [iOS apps](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) from the App Store (collected in July 2017). 

In [1]:
from csv import reader

opened_apple_store_file = open('/Users/lisacondoluci/Documents/AppleStore.csv')
read_apple_store_file = reader(opened_apple_store_file)
ios_data = list(read_apple_store_file)
opened_apple_store_file.close()
ios_header = ios_data[0]
ios_data = ios_data[1:]

opened_google_play_store_file = open('/Users/lisacondoluci/Documents/googleplaystore.csv')
read_google_play_store_file = reader(opened_google_play_store_file)
android_data = list(read_google_play_store_file)
opened_google_play_store_file.close()
android_header = android_data[0]
android_data = android_data[1:]

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
 
print('iOS Data:')
print(ios_header)
explore_data(ios_data, 0, 5)
print('\n')
print('Android Data:')
print(android_header)
explore_data(android_data, 0, 5)

iOS Data:
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']
['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']
['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']
['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']
['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Android Data:
['App', 'Category', 'Rating', 'Reviews', 'Si

After briefly exploring the data, the next step is to clean the data. I will remove any apps that are not *free*, and are not directed toward an *English-speaking* audience. 

In [2]:
print(android_data[10472])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


This row in the Google Play Store data has an error since the rating is 19, and the rating can only go up to 5 on Google Play Store, so I will delete it.

In [3]:
del android_data[10472]

Now I will check for duplicate entries in Google Play Store data. I'll determine which duplicate entry has the highest number of reviews, which would imply it is the most recent entry. Then, I will remove all other entries for a given app.

In [4]:
duplicate_apps = []
unique_apps = []

for app in android_data:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:15])
print('\n')
print('Actual expected length:', len(android_data) - len(duplicate_apps))

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


Actual expected length: 9659


Next, in order to remove all duplicate rows, I will create a data dictionary that includes the name of each app, as well as the highest number of reviews of all duplicate entries. When this is completed, I will check if the length of this data dictionary is the same as the actual expected length above, which it is.

In [5]:
reviews_max = {}

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    if name not in reviews_max:
        reviews_max[name] = n_reviews

len(reviews_max)

9659

The next step is to put all of the cleaned entries into a list of lists, called `android_clean`.

In [6]:
android_clean = []
already_added = []

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)

According to the ASCII, the numbers corresponding to characters we commonly use in English are all in the range 0 to 127. Therefore, using the function `ord()` will help determine the numbers associated with each letter of a string (the app name). I will define a function `english(string)` that will input a string and determine if it is written in English. In order to prevent deleting apps incorrectly, I will only remove an app that has more than three characters outside of the ASCII range. I will test this function with 4 app names below.

In [7]:
def english(string):
    non_english = 0
    for letter in string:
        if ord(letter) > 127:
            non_english += 1 
    if non_english > 3:
        return False
    else:
        return True

print(english('Instagram'))
print(english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(english('Docs To Go™ Free Office Suite'))
print(english('Instachat 😜'))

True
False
True
True


I will now filter out any non-English apps from both data sets.

In [8]:
english_ios_apps = []
for app in ios_data:
    name = app[1]
    if english(name) == True:
        english_ios_apps.append(app)

english_android_apps = []
for app in android_clean:
    name = app[0]
    if english(name) == True:
        english_android_apps.append(app)   
        
explore_data(english_ios_apps, 1, 5)
print('\n')
explore_data(english_android_apps, 1, 5)
print('\n')
print('Number of English iOS Apps:', len(english_ios_apps))
print('Number of English Android Apps:', len(english_android_apps))

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']
['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']
['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']
['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']
['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']
['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967

The final cleaning step is to isolate the free apps in a separate list.

In [9]:
final_ios = []
for app in english_ios_apps:
    price = app[4]
    if float(price) == 0.0:
        final_ios.append(app)
    
final_android = []
for app in english_android_apps:
    price = app[7]
    if price == '0':
        final_android.append(app)    

explore_data(final_ios, 1, 5)
print('\n')
explore_data(final_android, 1, 5)

print('\n')
print('Number of Free English iOS Apps:', len(final_ios))
print('Number of Free English Android Apps:', len(final_android))
        

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']
['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']
['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']
['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']
['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']
['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967

My validation strategy for an app idea is as follows:
1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, develop it further.
3. If the app is profitable after six months, build an iOS version of the app and add it to the App Store.

To analyze the app data, I will build frequency tables. To do this, I will build one function to generate frequency tables that show percentages, and another function to display percentages in a descending order using the `sorted()` function. To do this, I will have to transform the dictionary into a list of tuples, with each one containing a dictionary key and its corresponding value. The value must come first in order to sort. 

The `display_table()` function below takes in two parameters, `dataset` (a list of lists) and `index` (an integer). It generates a frequency table using the function `freq_table()`, transforms tje frequency table into a list of tuples, sorts the list in descending order, and prints the table.  

In [10]:
def freq_table(dataset, index):
    table = {}
    for app in dataset:
        genre = app[index]
        if genre in table:
            table[genre] += 1
        else:
            table[genre] = 1
    for genre in table:
        table[genre] = (table[genre] / len(dataset)) * 100
    return table        

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
    table_sorted = sorted(table_display, reverse=True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

I will start by examining the frequency table for the iOS apps using the `prime_genre` column. 

In [11]:
display_table(final_ios, 11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Among the free English apps on iOS, the highest frequency belongs to the category "Games", which is 58.16% of apps. Apps in the category "Entertainment" have a much lower frequency of 7.88%. The third most frequent category is "Photo & Video" at 4.97%. 

I will now analyze the Android apps, first using the Category column.

In [12]:
display_table(final_android, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The frequencies are quite different for the Android apps. The most frequent category is "Family", which has a percentage of 18.91%, while the next highest category, "Games", falls at just 9.72%. About 10% for Android compared to almost 60% for iOS for games is very interesting. To investigate further, we can see that the "Family" category actually does have several games, mainly for children. Therefore, the actual "Game" frequency may be closer to the sum of the highest 2 categories, about 30%. However, we can determine that the App Store seems to be dominated by gaming and entertainment apps, whereas the Google Play store tends to have a more balanced selection.

Additionally, there are more Android categories than iOS genres, which may be why the frequencies are more dispersed.

Next, I will analyze the Android apps using the Genre column.

In [13]:
display_table(final_android, 9) 

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

We can see that there are many more options for Genre on the Android data, so we will be using the Category data moving forward. 

The information we have at the moment does not provide enough to recommend an app profile, and the frequency tables do not necessarily reveal which genres have the most users. 

In order to determine which apps have the most users, I will analyze the number of `Installs` on the Android apps. This information is not available for the iOS apps, so I will be using the total number of user ratings as a proxy (column `rating_count_tot`). 

In [14]:
freq_ios = freq_table(final_ios, 11)
unique_genres = []
for row in freq_ios:
    unique_genres.append(row)
print(unique_genres)

['Social Networking', 'Photo & Video', 'Games', 'Music', 'Reference', 'Health & Fitness', 'Weather', 'Utilities', 'Travel', 'Shopping', 'News', 'Navigation', 'Lifestyle', 'Entertainment', 'Food & Drink', 'Sports', 'Book', 'Finance', 'Education', 'Productivity', 'Business', 'Catalogs', 'Medical']


In [15]:
for genre in unique_genres:
    total = 0   # store the sum of user ratings 
    len_genre = 0   # number of apps specific to each genre
    for row in final_ios:
        genre_app = row[11]
        if str(genre_app) == str(genre):
            user_ratings = int(row[5])
            total += user_ratings
            len_genre += 1
            
    avg_num_rat = total / len_genre
    print(genre, ':', avg_num_rat)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


We can see that apps in the "Navigation" genre have the  greatest number of user ratings. Next to "Navigation" is the "Reference" genre, and "Social Networking" comes in third place. For free, English apps on iOS, I recommend a Social Networking genre. 

In [16]:
display_table(final_android, 5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


We can see above that the `Installs` column of the Android table does not show precise numbers, but rather open ended ones. I will consider an app with 100,000+ installs as having 100,000. To do so, I will have to convert each of these strings to a float, meaning I have to remove the commas and the plus character. Below is an example of how I will do this.

In [17]:
n_installs = '100,000+'
n_installs = n_installs.replace('+', '')
n_installs = n_installs.replace(',', '')
print(n_installs)

100000


Now, I will calculate the average number of installs per app genre for the Google Play data set. 

In [18]:
freq_android = freq_table(final_android, 1)
unique_categories = []
for row in freq_android:
    unique_categories.append(row)
print(unique_categories)

['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY', 'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION', 'DATING', 'EDUCATION', 'ENTERTAINMENT', 'EVENTS', 'FINANCE', 'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME', 'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'GAME', 'FAMILY', 'MEDICAL', 'SOCIAL', 'SHOPPING', 'PHOTOGRAPHY', 'SPORTS', 'TRAVEL_AND_LOCAL', 'TOOLS', 'PERSONALIZATION', 'PRODUCTIVITY', 'PARENTING', 'WEATHER', 'VIDEO_PLAYERS', 'NEWS_AND_MAGAZINES', 'MAPS_AND_NAVIGATION']


In [19]:
for category in unique_categories:
    total = 0   # store the sum of installs per category
    len_category = 0   # number of apps specific to each category
    for row in final_android:
        category_app = row[1]
        if str(category_app) == str(category):
            user_installs = row[5]
            user_installs = user_installs.replace('+', '')
            user_installs = user_installs.replace(',', '')
            user_installs = int(user_installs)
            total += user_installs
            len_category += 1
            
    avg_num_cat = total / len_category
    print(category, ':', avg_num_cat)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

It looks as though the "Communication" category has the greatest number of installs, with "Social" being next. The app profile I recommend is "Communication".